# testing the routine on 1 file downloaded by google
Tim Tyree
1.21.2021

In [1]:
from lib.my_initialization import *
from lib.utils.utils_traj import *
import trackpy
from scipy import stats
from lib.utils.operari import *


%autocall 1
%load_ext autoreload
%autoreload 2
# awareness&chill

Automatic calling is: Smart


In [2]:
#make plots in darkmode
import seaborn as sns
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

# define helper functions

In [1]:
from worker.lib import *
from worker.lib.routines.generate_tip_logs_LR_model_cy import *
from worker.lib.routines.kwargs_LR_model_cy import get_kwargs
from worker.lib.utils.get_txt import get_txt
from worker.lib.routines.dag_log_to_msd import *
from worker.lib.routines.compute_msd import get_longest_trajectories

def log_to_unwrapped_trajectory(input_file_name, use_cache=True, sr=400, mem=0, **kwargs):
    '''ic is a .csv file name of a tip log.'''
    traj_fn = os.path.abspath(input_file_name).replace('/Log','/trajectories').replace('log.csv', f'traj_sr_{sr}_mem_{mem}.csv')
    output_file_name = traj_fn.replace('/trajectories','/trajectories_unwrap').replace('.csv',"_unwrap.csv")
    if not use_cache or not os.path.exists(traj_fn):
        traj_fn = generate_track_tips_pbc(input_file_name, save_fn=traj_fn, **kwargs)
    if not use_cache or not os.path.exists(output_file_name):
        retval_ignore = unwrap_trajectories(traj_fn, output_file_name)      
    return os.path.abspath(output_file_name)

# make function that handles remote task
- DONE: combine all cells related to main
- DONE: move import lines to front
- DONE: move parameters to front
- TODO: make all lib calls work from worker.lib
- TODO: test routine

and then, (consider running)
- TODO: wrap all this in my old ht condor boilerplate

In [2]:
# get_kwargs(ic_fn, results_folder=results_folder)

In [4]:
worker_dir=os.path.join(nb_dir,'worker')
width=412;height=412;
txt_id1=3;txt_id2=3

tmax_sec=.15 #max time to integratein seconds
# K_o=7.#5.4 higher K_o should give shorter APD#
dsdpixel=0.025#cm/pixel  # area=width*height*dsdpixel**2
dt=0.1# milliseconds
DT = 2#ms between spiral tip frames
save_every_n_frames=int(DT/dt)
tmin=100# milliseconds
sr=3600; mem=0;
n_tips = 1
round_t_to_n_digits=0


################################
# Setup file system and initial conditions
################################
results_folder=f'results'
ic_fn=os.path.join(worker_dir,f'ic-in/ic{width}x{height}.{txt_id1}.{txt_id2}.npz')
# param_fn = 'param_set_8_og.json'

#download and chunk initial conditions
os.chdir(worker_dir)
if not os.path.exists('ic-in'):
    os.mkdir('ic-in')
if not os.path.exists('ic-out'):
    os.mkdir('ic-out')
txt= get_txt(txt_id1,txt_id2,width,height,worker_dir)
np.savez_compressed(ic_fn,txt)
    
#initialize filesystem if not already initialized
cwd=os.getcwd()
# base_folder   = '/'+os.path.join(*cwd.split('/')[:-1])
base_folder=worker_dir
try:
    init_filesystem_bd(base_folder,
                      results_folder=results_folder,
                      subfolder_list=None)
except FileExistsError:
    print('new file system not initialized.')

#reset in out if ic-in is empty of npz files
enable_reset_in_out=True
if enable_reset_in_out:
    os.chdir(base_folder)
    def is_npz(s): return s[-4:]=='.npz'
    retval = [fn for fn in os.listdir('ic-in') if is_npz(fn)]
    if len(retval)==0:
        os.rename('ic-in','ic-in2')
        os.rename('ic-out','ic-in')
        os.rename('ic-in2','ic-out')
        print('ic reset')

################################
# Generate tip logs
################################
def routine(ic):
#     kwargs = get_kwargs(ic)   
    kwargs = get_kwargs(ic, results_folder=results_folder)#,param_fn=param_fn)
#     kwargs['V_threshold'] =  -50.#mV
    kwargs['dsdpixel']=dsdpixel
    kwargs['h']=dt# kwargs['h']=0.01 for ds_1_param_set_8 for stability
    kwargs['tmax_sec'] = tmax_sec #maximum time to be integrated in seconds
    kwargs['tmax']= tmax_sec*10**3
    kwargs['tmin']= tmin #millisecondds
    kwargs['save_every_n_frames']=save_every_n_frames
    #     kwargs['K_o']=K_o#TODO: expose K_o to this level if it should be varied extensively
    kwargsout = generate_tip_logs_from_ic(ic, **kwargs)
    #     print(f"completed birth_death_rates_from_ic: {ic}")
    #     print(f"csv of spiral tip data stored in: {kwargs['completed_ic_dir']}")
    #     return os.path.basename(ic)
    return kwargsout#['data_dir_log']#output location


#get all .npz files in the file's directory
input_fn_lst=get_all_files_matching_pattern(file=ic_fn, trgt='.npz')
assert ( len(input_fn_lst)>0)

#generate tip logs
retval=[routine(ic) for ic in input_fn_lst]
log_dir=os.path.join(worker_dir,results_folder,'Log',retval[0]['data_fn_log'])
# #generate tip logs with multiple threads in parallel
# b = db.from_sequence(input_fn_lst, npartitions=1).map(routine)
# start = time.time()
# retval = list(b)

################################
# Track, unwrap, and select longest trajectories
################################
unwrapped_fn=log_to_unwrapped_trajectory(log_dir, use_cache=True, 
                                         sr=sr, mem=mem)
#get longest unwrapped trajectory
df=get_longest_trajectories(unwrapped_fn, 
                            n_tips = n_tips, DS = dsdpixel,DT = DT, 
                            round_t_to_n_digits=round_t_to_n_digits)
#save .csv in log
os.chdir(worker_dir)
if not os.path.exists('Log'):
    os.mkdir('Log')
df.to_csv('Log/out.csv')

new file system not initialized.


In [6]:
#print to stdout
with open('Log/out.csv') as f:
    for line in f:
        print(line)
#     fh.close()

,index,t,n,x,y,grad_ux,grad_uy,grad_vx,grad_vy,frame,particle

669,669,100.0,38,0.0,0.0,-0.01576,1.42445,0.33624,1.27909,0,32

670,670,102.0,36,-0.07239000000004125,0.03095999999999321,-0.016819999999999998,1.42487,0.33321999999999996,1.28035,1,32

671,671,104.0,40,-0.16550000000003706,0.0633200000000329,-0.01788,1.4252799999999999,0.33035,1.28155,2,32

672,672,106.0,46,-0.2784100000000365,0.09769000000000004,-0.018869999999999998,1.42565,0.3277,1.2826600000000001,3,32

673,673,108.0,46,-0.4411700000000565,0.13301000000001295,-0.041139999999999996,1.43241,0.20269,1.33307,4,32

674,674,110.0,46,-0.6771500000000401,0.1693000000000211,-0.04034,1.43204,0.20185999999999998,1.33342,5,32

675,675,112.0,46,-0.9618800000000364,0.2102700000000368,-0.03882,1.43135,0.20338,1.33279,6,32

676,676,114.0,46,-1.3027200000000505,0.2586699999999951,-0.03635,1.4302700000000002,0.2071800000000001,1.3312,7,32

677,677,116.0,46,-1.8909000000000449,0.3006700000000251,-0.07049,1.44182,0.1009,1.3732799999999998

In [77]:
# #test the V_threshold value
# j=0
# V_threshold=-50.
# level1 = V_threshold

# t=-999
# level2 = 0.
# ic=input_fn_lst[j]
# txt=load_buffer(ic)#input_fn)
# inVc,outVc,inmhjdfx,outmhjdfx,dVcdt=unstack_txt(txt)
# img=inVc[...,0]
# dimgdt=dVcdt[...,0]
# compute_all_spiral_tips= get_compute_all_spiral_tips(mode='simp')#width=width,height=height)
# dict_out=compute_all_spiral_tips(t,img,dimgdt,level1,level2)#,width=width,height=height)
# print(len(list(dict_out['x'])))
# fig=show_buffer_LR(txt)
# axs=fig.get_axes()
# ax=axs[0]
# ax.scatter(dict_out['x'],dict_out['y'],s=1500,c='yellow',marker='*')
# plt.show()